In [4]:
import pandas as pd 

url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle/3004/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1558031593&Signature=D4Vjhmewf54tgRRJCteAzCrDqp%2F3%2FntnkHKiU6pv9wjBStnooszSaRFOr0K6XWvaC7XRChk0LQ1i9UbbyVyjqSDmLU9w%2Fx5DlEShkaYhgXCp9H9sr6EFFUHnrhDn5z8%2FOQKhO7weEMaZSJPClZkPpNtwjJihdkX3atQSmPP5Iab1DJq71XBJ4wkExB8g%2BWL0Cs7pfr5Tkil4PQcQIJsw6yQ3cIX9lXv5uY9%2FDmCQGySvXTkHdoNWRR%2FeXL5VTdwuVx1vjiIup0zv%2F%2FA%2FacvR7%2FIPiikinOGRQ7yphsqnS9icboFkt%2FKGsbgsZhKaGQENXPqbJZtYuxnJMVUwwA%2BCcg%3D%3D"
df = pd.read_csv(url)


In [5]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,1:].values
y = df.iloc[:,0].values

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                test_size = 0.3,
                                                random_state = 0)

In [12]:
mean_vals = np.mean(x_train, axis=0)
std_val = np.std(x_train)

x_train_centered = (x_train - mean_vals)/std_val
x_test_centered = (x_test - mean_vals)/std_val

In [6]:
import numpy as np
def batch_generator(x,y,batch_size = 64,shuffle = False, random_seed = None):
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        x = x[idx]
        y = y[idx]
        
    for i in range(0,x.shape[0],batch_size):
        yield (x[i:i+batch_size,:], y[i:i+batch_size])      

In [7]:
mean_vals = np.mean(x_train, axis=0)
std_val = np.std(x_train)

x_train_centered = (x_train - mean_vals)/std_val
x_test_centered = (x_test - mean_vals)/std_val

In [8]:
import tensorflow as tf 
import numpy as np 

def conv_layer(input_tensor,name,kernel_size,n_output_channels,
               padding_mode ="SAME",strides = (1,1,1,1)):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        
        weights_shape = list(kernel_size) + [n_input_channels,n_output_channels]
        weights = tf.get_variable(name = "weights",shape = weights_shape)
        print(weights)

        biases = tf.get_variable(name = "_biases",
                                 initializer = tf.zeros(
                                     shape = [n_output_channels]))
        
        print(biases)
        
        conv = tf.nn.conv2d(input = input_tensor,
                            filter = weights,
                            strides = strides,
                            padding = padding_mode)
        
        print(conv)
        
        conv = tf.nn.bias_add(conv,biases,name = "net_pre-activation")
        
        print(conv)
        
        conv = tf.nn.relu(conv,name = "activation")
        print(conv)
        
        return conv
        
        
def fc_layer(input_tensor,name, n_output_units,activation_fn = None):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:]
        n_input_units = np.prod(input_shape)
        
        if len(input_shape) > 1:
            input_tensor = tf.reshape(input_tensor,
                                      shape = (-1,n_input_units))
            
        weights_shape = [n_input_units,n_output_units]
        weights = tf.get_variable(name = "_weights",shape = weights_shape)
        print(weights)
        
        
        
        biases = tf.get_variable(name = "_biases",
                                 initializer = tf.zeros(shape = [n_output_units]))  
        print(biases)
        
        
        layer = tf.matmul(input_tensor,weights)
        print(layer)
        
        layer = tf.nn.bias_add(layer, biases,
                              name='net_pre-activaiton')
        
        print(layer)
        
        if activation_fn is None:
            return layer
        
        layer = activation_fn(layer,name="activation")
        print(layer)
        
        return layer        

In [9]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32,shape = [None,28,28,1])
    conv_layer(x,name = "convtest",kernel_size = (3,3), n_output_channels = 32)
    

g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, activation_fn=tf.nn.relu)
    
        

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'convtest/weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
<tf.Variable 'convtest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("convtest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(?, 28, 28, 32), dtype=float32)
<tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
<tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/net_pre-activaiton:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [18]:
import os 
def build_cnn(learning_rate=1e-4):
    ## Placeholders for X and y:
    tf_x = tf.placeholder(tf.float32, shape=[None, 784],
                          name='tf_x')
    
    tf_y = tf.placeholder(tf.int32, shape=[None],
                          name='tf_y')

    # reshape x to a 4D tensor: 
    # [batchsize, width, height, 1]
    tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1],
                            name='tf_x_reshaped')
    ## One-hot encoding:
    tf_y_onehot = tf.one_hot(indices=tf_y, depth=10,
                             dtype=tf.float32,
                             name='tf_y_onehot')

    ## 1st layer: Conv_1
    print('\nBuilding 1st layer:')
    h1 = conv_layer(tf_x_image, name='conv_1',
                    kernel_size=(5, 5), 
                    padding_mode='VALID',
                    n_output_channels=32)
    ## MaxPooling
    h1_pool = tf.nn.max_pool(h1, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')
    ## 2n layer: Conv_2
    print('\nBuilding 2nd layer:')
    h2 = conv_layer(h1_pool, name='conv_2', 
                    kernel_size=(5, 5), 
                    padding_mode='VALID',
                    n_output_channels=64)
    ## MaxPooling 
    h2_pool = tf.nn.max_pool(h2, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')

    ## 3rd layer: Fully Connected
    print('\nBuilding 3rd layer:')
    h3 = fc_layer(h2_pool, name='fc_3',
                  n_output_units=1024, 
                  activation_fn=tf.nn.relu)

    ## Dropout
    keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')
    h3_drop = tf.nn.dropout(h3, keep_prob=keep_prob, 
                            name='dropout_layer')

    ## 4th layer: Fully Connected (linear activation)
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop, name='fc_4',
                  n_output_units = 10, 
                  activation_fn = None)

    ## Prediction
    predictions = {
        'probabilities': tf.nn.softmax(h4, name='probabilities'),
        'labels': tf.cast(tf.argmax(h4, axis=1), tf.int32,
                           name='labels')
    }

    ## Loss Function and Optimization
    cross_entropy_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=h4, labels=tf_y_onehot),
        name='cross_entropy_loss')

    ## Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss,
                                   name='train_op')
    ## Computing the prediction accuracy
    correct_predictions = tf.equal(
        predictions['labels'], 
        tf_y, name='correct_preds')

    accuracy = tf.reduce_mean(
        tf.cast(correct_predictions, tf.float32),
        name='accuracy')

def save(saver,sess,epoch,path = "./model/"):
    if not os.path.isdir(path):
        os.makedirs(path)
    print("Saving model in %s" % path)
    saver.save(sess,os.path.join(path,"cnn-model.ckpt"),
              global_step = epoch)
    
def load(saver,sess,path,epoch):
    print('Loading model from %s' % path)
    saver.restore(sess,os.path.join(
    path,"cnn-model.ckpt-%d" %epoch))
    
def train(sess, training_set, validation_set = None,
         initialize = True, epochs = 20 , shuffle = True,
          dropout = 0.5, random_seed = None):
    
    x_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []
    
    if initialize:
        sess.run(tf.global_variables_initializer())
        
    np.random.seed(random_seed)
    
    for epoch in range(1,epochs + 1):
        batch_gen = batch_generator(
        x_data, y_data, shuffle = shuffle)
        
        avg_loss = 0.0
        for i, (batch_x,batch_y) in enumerate(batch_gen):

            feed = {"tf_x:0":batch_x,
                    "tf_y:0":batch_y,
                    "fc_keep_prob:0" : dropout }

            loss, _ = sess.run(
            ["cross_entropy_loss:0","train_op"],
                feed_dict = feed)

            avg_loss += loss
        training_loss.append(avg_loss / (i + 1))
        
        print('Epoch %02d Training Avg. Loss: %7.3f' % (
            epoch, avg_loss), end=' ')
        
        if validation_set is not None:
            feed = {
                "tf_x:0": validation_set[0],
                "tf_y:0": validation_set[1],
                "fc_keep_prob:0" : 1.0 }
            
            valid_acc = sess.run("accuracy:0",feed_dict = feed)
            print(' Validation Acc: %7.3f' % valid_acc)
            
        else:
            print()
            
def predict(sess,x_test,return_proba = False):
    feed = {
        "tf_x:0":x_test,
        "fc_keep_proba:0":1.0
    }
    
    if return_proba:
        return sess.run("probabilities:0",feed_dict = feed)
    else:
        return sess.run("labels:0",feed_dict = feed)
    
        
        
    
    

In [17]:
random_seed = 123

g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    build_cnn()
    
    saver = tf.train.Saver()
    
with tf.Session(graph = g) as sess:
    train(sess,
         training_set = (x_train_centered,y_train),
         validation_set = (x_test_centered, y_test),
         initialize = True,
         random_seed = 123)
    




Building 1st layer:
<tf.Variable 'conv_1/weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer:
<tf.Variable 'conv_2/weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activaiton:0", shape=(?, 1024), dtype=float32)
Te

NameError: name 'os' is not defined

In [ ]:
del g 

g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    build_cnn()
    saver = tf.train.Saver()
    
with tf.Session(graph = g) as sess:
    load(saver,sess,epoch=20,path = "./model/")
    
    preds = predict(sess,x_test_centered,return_proba = False)
    
    print('Test Accuracy: %.3f%%' % (100*np.sum(preds == y_test)/len(y_test)))
    
    print(predict(sess, X_test_centered[:10], 
            return_proba=False))
   
    print(predict(sess, X_test_centered[:10], 
                  return_proba=True))
   

In [ ]:
with tf.Session(graph = g) as sess:
    load(saver, sess, 
         epoch=20, path='./model/')
    
    train(sess,
         training_set = (x_train_centered,y_train),
         validation_set = (x_test_centered, y_test),
          initialize=False,
          epochs=20,
          random_seed=123)
        
    save(saver, sess, epoch=40, path='./model/')
    
    preds = predict(sess, X_test_centered, 
                    return_proba=False)
    
    print('Test Accuracy: %.3f%%' % (100*
                np.sum(preds == y_test)/len(y_test)))

In [ ]:
import tensorflow as tf
import numpy as np


class ConvNN(object):
    def __init__(self, batchsize=64,
                 epochs=20, learning_rate=1e-4, 
                 dropout_rate=0.5,
                 shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
                
        g = tf.Graph()
        with g.as_default():
            ## set random-seed:
            tf.set_random_seed(random_seed)
            
            ## build the network:
            self.build()

            ## initializer
            self.init_op = \
                tf.global_variables_initializer()

            ## saver
            self.saver = tf.train.Saver()
            
        ## create a session
        self.sess = tf.Session(graph=g)
                
    def build(self):
        
        ## Placeholders for X and y:
        tf_x = tf.placeholder(tf.float32, 
                              shape=[None, 784],
                              name='tf_x')
        tf_y = tf.placeholder(tf.int32, 
                              shape=[None],
                              name='tf_y')
        is_train = tf.placeholder(tf.bool, 
                              shape=(),
                              name='is_train')

        ## reshape x to a 4D tensor: 
        ##  [batchsize, width, height, 1]
        tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1],
                              name='input_x_2dimages')
        ## One-hot encoding:
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=10,
                                 dtype=tf.float32,
                                 name='input_y_onehot')

        ## 1st layer: Conv_1
        h1 = tf.layers.conv2d(tf_x_image, 
                              kernel_size=(5, 5), 
                              filters=32, 
                              activation=tf.nn.relu)
        ## MaxPooling
        h1_pool = tf.layers.max_pooling2d(h1, 
                              pool_size=(2, 2), 
                              strides=(2, 2))
        ## 2n layer: Conv_2
        h2 = tf.layers.conv2d(h1_pool, kernel_size=(5, 5), 
                              filters=64, 
                              activation=tf.nn.relu)
        ## MaxPooling 
        h2_pool = tf.layers.max_pooling2d(h2, 
                              pool_size=(2, 2), 
                              strides=(2, 2))

        ## 3rd layer: Fully Connected
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, 
                              shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024, 
                             activation=tf.nn.relu)

        ## Dropout
        h3_drop = tf.layers.dropout(h3, 
                              rate=self.dropout_rate,
                              training=is_train)
        
        ## 4th layer: Fully Connected (linear activation)
        h4 = tf.layers.dense(h3_drop, 10, 
                             activation=None)

        ## Prediction
        predictions = {
            'probabilities': tf.nn.softmax(h4, 
                              name='probabilities'),
            'labels': tf.cast(tf.argmax(h4, axis=1), 
                              tf.int32, name='labels')
        }
        
        ## Loss Function and Optimization
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h4, labels=tf_y_onehot),
            name='cross_entropy_loss')
        
        ## Optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss,
                              name='train_op')

        ## Finding accuracy
        correct_predictions = tf.equal(
            predictions['labels'], 
            tf_y, name='correct_preds')
        
        accuracy = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32),
            name='accuracy')

    def save(self, epoch, path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        print('Saving model in %s' % path)
        self.saver.save(self.sess, 
                        os.path.join(path, 'model.ckpt'),
                        global_step=epoch)
        
    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess,
             os.path.join(path, 'model.ckpt-%d' % epoch))
        
    def train(self, training_set, 
              validation_set=None,
              initialize=True):
        ## initialize variables
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])

        for epoch in range(1, self.epochs+1):
            batch_gen = \
                batch_generator(X_data, y_data, 
                              shuffle=self.shuffle)
            avg_loss = 0.0
            for i, (batch_x,batch_y) in \
                enumerate(batch_gen):
                feed = {'tf_x:0': batch_x, 
                        'tf_y:0': batch_y,
                        'is_train:0': True} ## for dropout
                loss, _ = self.sess.run(
                        ['cross_entropy_loss:0', 'train_op'], 
                        feed_dict=feed)
                avg_loss += loss
                
            print('Epoch %02d: Training Avg. Loss: '
                  '%7.3f' % (epoch, avg_loss), end=' ')
            if validation_set is not None:
                feed = {'tf_x:0': batch_x, 
                        'tf_y:0': batch_y,
                        'is_train:0' : False} ## for dropout
                valid_acc = self.sess.run('accuracy:0',
                        feed_dict=feed)
                print('Validation Acc: %7.3f' % valid_acc)
            else:
                print()
                    
    def predict(self, X_test, return_proba=False):
        feed = {'tf_x:0' : X_test,
                'is_train:0' : False} ## for dropout
        if return_proba:
            return self.sess.run('probabilities:0',
                                 feed_dict=feed)
        else:
            return self.sess.run('labels:0',
                                 feed_dict=feed)